In [1]:
import utils

In [2]:
LOWER = True
NORM_TENSE = True
REP_NUM = True
REP_YEAR = True
REP_SYM = False

In [3]:
dataset_folder = "data/AL/"
train_data = dataset_folder + "train"
lines = utils.read_file_to_lines(train_data)

In [4]:
sentences = []
pos_tags = []

while len(lines) > 1:
    sentence_break = lines.index("")
    sentence_xy = lines[:sentence_break]
    words = [utils.preprocess_al_text(token.split(" ")[0],
                                      lower=LOWER, replace_number=REP_NUM)
             for token in sentence_xy]
    pos = [token.split(" ")[1] for token in sentence_xy]
    sentences.append(words)
    pos_tags.append(pos)
    lines = lines[sentence_break+1:]

In [5]:
name_categories = [
    'CITY', 'COMMUNITY', 'COUNTRY', 'DISTRICT', 'POI', 'PROV', 'REDUNDANT', 'ROAD', 'TOWN'
]

In [6]:
name_words = []

def get_entities(sentence, pos_tag):
    for i in range(len(sentence)):
        try:
            pos = pos_tag[i].split("-")[1]
            word = sentence[i]
            if pos in name_categories:
                if len(word) < 2:
                    name_words.append(word)
        except Exception as e:
            pass
        
for n in range(len(sentences)):
    sentence = sentences[n]
    pos_tag = pos_tags[n]
    get_entities(sentence, pos_tag)
    
def clean_list(input_list):
    input_list = list(set(input_list))
    input_list.sort()
    return input_list

name_words = clean_list(name_words)
len_name_words = len(name_words)

In [7]:
import copy
from random import randint, choice

In [8]:
def rand_bool():
    return choice([True, False])

def augment_line(sentence, pos_tag):
    start = False
    for i, word in enumerate(sentence):
        try:
            b, pos = pos_tag[i].split("-")
            if pos in name_categories:
                r_i = randint(0, len_name_words)
                sentence[i] = name_words[r_i]
        except:
            pass
    return sentence

In [9]:
aug_lines = []

#for n in range(len(sentences)):
for n in range(20):
    sentence = sentences[n]
    sentence = copy.deepcopy(sentence)
    print("org:", " ".join(sentence))
    pos_tag = pos_tags[n]
    sentence = augment_line(sentence, pos_tag)
    for i, word in enumerate(sentence):
        pos = pos_tag[i]
        line = word + " " + pos
        aug_lines.append(line)
    print("aug:", " ".join(sentence))
    print("")

org: 灏 娓 鍥 鏂 瀹 鍥 1-4 骞 1-6 瀹
aug: 鐏 璁 绂 姘 瀣 钖 1-4 骞 1-6 瀹

org: 鍢 鍏 鍗 婀 鍖 鏂 鏄 璺 鏂 鏂 鍖 骞 鍦 鐢 鑱
aug: - 婧 鎼 寰 璺 绯 閾 m 鐖 鎸 楗 闃 鍞 楂 閱

org: 娴 姹 鐪 瀹 娉 甯 娴 鏇 鍖 鐧 鏉 琛 4-4 寮 1-5 鍙 闃 鍏 澶 鑽 鎴
aug: 瀹 鐩 鑻 鍜 骞 濞 涓 娓 h 閴 鎽 鎷 4-4 寮 1-5 鍙 纭 鎰 鐥 瑜 鍗

org: 娴 姹 鐪 瀹 娉 甯 闀 娴 鍖 闀 娴 鍖 閽 鍖 璺 3-3 鍙 鍚 鐢 鍥 闄 鍩 6-5 骞 鏋 绌 灞 涓 宸
aug: 鍩 妗 澧 浼 妲 娌 鐩 瑜 鐭 鐭 鍎 鏉 鎯 姘 鏄 3-3 鍙 瀣 绁 缍 璺 鍜 6-5 骞 鏋 绌 灞 涔 铻

org: 娓 宸 涔 娓 鏌 甯 闀 铦 涓 鐗 娴 鍥 涔 涔 涓 绾
aug: 宕 鑽 鍓 鍍 g l 鐦 h 绫 j 鎼 鍥 涔 涔 涓 绾

org: 娴 姹 鐪 鍢 鍏 甯 娴 瀹 甯 鏂 妗 闀 鏃 灏 浜 涓 鍥 鍖 8-4 骞
aug: 鐘 鍧 钀 鍕 瑙 婢 浠 鑻 瓒 鏂 鏌 娼 瀚 钀 鍏 鍏 铓 鐒 8-4 骞

org: 娴 姹 鐪 鏉 宸 甯 浣 鏉 鍖 涓 骞 鍗 鑻 琛 閬 鍗 澶 琛 9-4 鍙 浣 鏉 鍐 鍟 浜 鍔 璧 婧 閮
aug: 铻 铦 鍑 閰 鐧 鍧 璞 杞 鍑 閬 妲 鐧 鍕 e 鐥 鍗 澶 琛 9-4 鍙 韬 椴 h 鍑 鐐 璋 宥 铏 璁

org: 鍜 涔 璺 1-5 鍙 姹 閲 澶 鍘 1-6
aug: 鐥 娆 鏌 1-5 鍙 搴 婀 楹 鍟 1-6

org: 娴 姹 鐪 婀 宸 甯 绾 鏃 璺 1-6 鍙 婀 宸 甯 涓 蹇 鍖 闄
aug: 鎻 鐧 娼 e 鑲 宸 绁 j 缂 1-6 鍙 灏 鏄 宀 瑗 u 缈 绡

org: 鑻 鍗 榫 娓 鍒 鍗 NUM-THING 鍙
aug: 宀 浼 婕 琚 閾 浣 NUM-THING 鍙

org: 涓 娴 甯 闂 鍖 鍖 娌 鍗 鍖 璺 1-6 鍙
aug: 妗 绂 鍧 鏋 u 楗 妯 瑷 f 鐏 1-6 鍙

org: 涓 娴 甯 娴 涓 鍖 搴 妗 闀 绉 娴 璺 3-6 寮 鍒 鐮 鏅 閫 1-5 鍙 d 1-4 妤 濞 绾 浼 

In [10]:
#aug_lines[:100]